In [ ]:
import import_ipynb

from matplotlib import pyplot as plt
from termcolor import colored
import cartopy.crs as ccrs
import matplotlib.colors as colors
import xarray as xr
import numpy as np
import cartopy
import matplotlib.gridspec as gridspec
from glob import iglob
from os.path import join
from functools import reduce
import pandas as pd
import harp 
import itertools
import cartopy.feature as cf
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.patches as mpatches
import imageio
import netCDF4
from netCDF4 import Dataset
import cartopy.io.shapereader as shpreader
import cartopy.io.img_tiles as cimgt
import matplotlib.pyplot as plt
import cartopy.mpl.geoaxes
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
%matplotlib inline
import numpy as np

In [ ]:
product_path = "C:/DELHI2020NO2JAN"


#2019 data stored in a list
files2019 =sorted(list(iglob(join(product_path, "*2019*.nc"), recursive =True)))
print(colored("NO2 2019 files:","red"), len(files2019))

#2020 data stored in a different list
files2020 = sorted(list(iglob(join(product_path, "*2020*.nc"), recursive =True)))
print(colored("NO2 2020 files:","red"), len(files2020))


#sample file for testing on a 1stMARCH2019 data
file1 = input_files_2020[0]
file2 = input_files_2020[1]
print(colored("Product selected for analyis:","blue"), file1)
print(colored("Product selected for analyis:","blue"), file2)

In [ ]:
#opening product ----Global Attributes
with xr.open_dataset(file1) as file1GA:
    print(colored("Global attributes:\n", "blue"), file1GA)


#opeining product METADATA
with xr.open_dataset(file1, group = "METADATA") as file1MT:
    print(colored("\nMETADATA:\n", "blue"), file1MT)
    
with xr.open_dataset(file1, group ="PRODUCT") as file1PRD:
    print(colored("\nPRODUCT GROUP:\n", "blue"), file1PRD)

In [ ]:
#DATA VARIABLES ACCESS
no2 = file1PRD['nitrogendioxide_tropospheric_column']
print(colored("\nAttributes:", "blue"), no2.attrs)
print(colored("\nValues:", "blue"), no2.values)
print(colored('Dimension of axis:', "blue"), no2.dims)
print(colored("\nCoordinates", "blue"), no2.coords)


In [ ]:
print(no2[0])

In [ ]:
### Converting values to standard units
### Filtering out data
no2 = no2*(6.02214e+19)
no2_filter = no2.where(file1PRD['qa_value'] > 0.75, drop = True)

### Average Data Collection

In [ ]:
no2_new = no2
out = np.nan_to_num(no2_new)  
print(out)
print(no2_new)
print(no2_new.size)
print(np.count_nonzero(no2_new))

print(np.sum(no2_new))

In [ ]:
### Obtaining data from all files
import sys
for i in files2020:
    file1PRD = xr.open_dataset(i, group ="PRODUCT")
    no2 = file1PRD['methane_mixing_ratio']
    no2_new = np.nan_to_num(no2.values)
    sumele = np.sum(no2_new)
    nonzero = np.count_nonzero(no2_new)
    average = sumele/nonzero
    name = i[50:58]
    print(name," ", average)
    

### Plots


In [ ]:
from IPython.display import IFrame
ne = IFrame(src='https://www.naturalearthdata.com', width=1000, height=500)
display(ne)

In [ ]:
plt.figure(figsize=(17,12))
ax = plt.subplot(111, projection=ccrs.Orthographic(central_latitude=20.7041, central_longitude=77.1025))
no2[0].plot.pcolormesh(ax=ax, x ='longitude', y='latitude', add_colorbar=True, cmap='magma_r',transform=ccrs.PlateCarree(),vmin=0)
ax.add_feature(cartopy.feature.RIVERS)
ax.set_title('NO2(Delhi) 2019/03/09')
ax.coastlines('10m')
###ADDING STATES TO THE GRAPH
states_provinces = cf.NaturalEarthFeature(category='cultural',
                                                name='admin_1_states_provinces',
                                                scale='10m',
                                                facecolor='none')

ax.add_feature(cf.LAND)
ax.add_feature(cf.COASTLINE)
ax.add_feature(states_provinces, edgecolor='gray')         
ax.add_feature(cf.LAND)
ax.add_feature(cf.OCEAN)
ax.add_feature(cf.COASTLINE)
ax.add_feature(cf.BORDERS,)
ax.add_feature(cf.LAKES, alpha=0.5 )
ax.add_feature(cf.RIVERS, alpha=1)
ax.stock_img()
ax.gridlines()

#Adding Delhi to our plot

ax.plot(28.5,77.5,'bo', markersize=7, transform=ccrs.PlateCarree())
ax.text(29, 78, "Delhi")
#Saving the plot
plt.savefig('C:/Users/resultmay/', bbox_inches='tight', dpi =300)



In [ ]:
plt.figure(figsize=(12, 12))
ax = plt.axes(projection=ccrs.PlateCarree())
#no2[0].plot.pcolormesh(ax=ax, x='longitude', y='latitude', add_colorbar = True, cmap='magma_r', vmin=0);


# natural earth
# remove ne_10m_ from ne_10m_admin_1_states_provinces downloaded file name for name argument assignment
states_provinces = cf.NaturalEarthFeature(category='cultural',
                                                name='admin_1_states_provinces',
                                                scale='10m',
                                                facecolor='none')

ax.add_feature(cf.LAND)
ax.add_feature(cf.COASTLINE)
ax.add_feature(states_provinces, edgecolor='gray')

### Quality Filter

In [ ]:
no2_filter = no2.where(file1PRD['qa_value'] > 0.75, drop = True)

#plot original data vs quality flag
plt.figure(figsize=(30, 10))
                
#Plot masked NO2 data                
ax2 = plt.subplot(122, projection=ccrs.Orthographic(11, 42))
no2_filter[0].plot.pcolormesh(ax=ax2, x='longitude', y='latitude', add_colorbar =True,cmap='magma_r',transform=ccrs.PlateCarree(), vmin=0)
ax2.set_tile('Filtered S_5p ,Quality Flag Layer> 0.75')
ax2.add_feature(cartopy.feature.RIVERS)
ax2.coastlines('10m')
ax2.stock_img()
ax2.gridlines()
                
### Save the file
                
plt.savefig("C:/Users/resultmay/",bbox_inches='tight', dpi=300)
                

### Delhi Subset

In [ ]:
ur = (29,77.5 )
ll = (28.3,76.8 )
no2_subset = no2_filter.where((no2_filter.longitude < ur[1]) & (no2_filter.longitude > ll[1]) & (no2_filter.latitude < ur[0]), drop = True)

#plot subset
plt.figure(figsize(12,12))
ax = plt.axes(projection=ccrs.Orthographir(11, 42))

stt_prv =cf.NaturalEarthFeature(category='cultural',name='admin_1_states_provinces_lines',scale='10m',facecolor='none')

no2_subset[0].plot.pcolormesh(ax=ax, x='longitude', y='latitude', add_colorbar=True, cmaps='magma_r',transform=ccrs.PlateCarree(), vmin=0)
ax.add_feature(cartopy.feature.LAND, edgecolors='black')
ax.add_feature(stt_prc, linewidth=0.2, edgecolor='black')
ax.set_title('NO2 concentration | qa_value >0.75')

### Add POI to map
ax.text(78, 29, 'Delhi', transform=ccrs.Geodetic());

###save figure to file
plt.savefig('/LOCATION')

In [ ]:
#WIDGETS LAYOUT

land = ipywidgets.Checkbox(description='land', style={'description_width': 'initial'})
ocean = ipywidgets.Checkbox(description='ocean', style={'description_width': 'initial'})
coastline = ipywidgets.Checkbox(description='coastline', style={'description_width': 'initial'})
borders = ipywidgets.Checkbox(description='borders', style={'description_width': 'initial'})
lakes = ipywidgets.Checkbox(description='lakes', style={'description_width': 'initial'})
rivers = ipywidgets.Checkbox(description='rivers', style={'description_width': 'initial'})
states = ipywidgets.Checkbox(description='states', style={'description_width': 'initial'})
plot_locations = ipywidgets.Checkbox(description='plot_locations', style={'description_width': 'initial'})

box_layout = ipywidgets.Layout(display='inline-flex',                               
                               flex_flow='row',
                               align_items='stretch',
                               border='solid',
                               width='100%')

ui = ipywidgets.HBox([land, ocean, coastline, borders, lakes, rivers,states, plot_locations], layout=box_layout)

def create_map(land,ocean,coastline,borders,lakes,rivers,states,plot_locations):
    data_crs = ccrs.PlateCarree()
    ax = plt.figure(figsize=(12, 8))
    #ax = ax.add_subplot(1, 1, 1, projection=data_crs)
    #ax.set_extent([68, 97, 7, 35], ccrs.PlateCarree())  
    #STATESLAYOUTDEFINATION
    states_provinces = cf.NaturalEarthFeature(category='cultural',
                                                name='admin_1_states_provinces',
                                                scale='10m',
                                                facecolor='none')

    ###Global Map
    #ax.set_global()
    if land == True: ax.add_feature(cfeature.LAND)
    if ocean == True: ax.add_feature(cfeature.OCEAN)
    if coastline == True: ax.add_feature(cfeature.COASTLINE)
    if borders == True: ax.add_feature(cfeature.BORDERS)
    if lakes == True: ax.add_feature(cfeature.LAKES, alpha=0.5)
    if rivers == True: ax.add_feature(cfeature.RIVERS)
    if states == True: ax.add_feature(cfeature.LAND),ax.add_feature(cfeature.COASTLINE),ax.add_feature(states_provinces, edgecolor='gray')    
    if plot_locations == True:
        delhi_lon, delhi_lat = 77.23, 28.61
        ax.plot(delhi_lon, delhi_lat, 'o', transform=data_crs)
        ax.text(delhi_lon + 1, delhi_lat - 6, 'Delhi', horizontalalignment='left', transform=data_crs)
        
out = ipywidgets.interactive_output(create_map, {'land': land,
                                                 'ocean': ocean,
                                                 'coastline': coastline,
                                                 'borders': borders,
                                                 'lakes': lakes,
                                                 'rivers': rivers,
                                                 'states': states,
                                                 'plot_locations': plot_locations})

display(ui, out)